# How robo-advisors make investment decisions

The examples below demonstrate the basic concept of how robo-advisors make investment decisions:

- Basic portfolio optimization using Modern Portfolio Theory (covered in much greater detail last time)
- Tax-loss harvesting logic
- **Risk profiling and asset allocation** (THIS FILE)
- Portfolio rebalancing

We will use these examples to:

- Demonstrate how robo-advisors make investment decisions
- Show how risk profiles are translated into portfolio allocations
- Illustrate tax-loss harvesting logic
- Demonstrate portfolio rebalancing mechanics

Each example can be expanded. You may want to try:

- Add additional (or interactive) visualization using `matplotlib` or `plotly`
- Include more sophisticated optimization techniques (think `riskfolio` package)
- Add error handling and input validation (useful for more advanced and user-friendly applications)
- Include more complex tax rules
- Add additional real-time market data integration (more stocks, different time periods, alternative assets).

# Understanding Robo-Advisor Risk Assessment Example

This example demonstrates how robo-advisors assess client risk profiles and translate them into investment recommendations. The process involves collecting client information through questionnaires, calculating risk scores, and mapping these scores to appropriate asset allocations.

Often, during new client registration on robo-platforms, clients fill out a form known as **KYC form** (*Know Your Client* form).

## Key Components

### 1. Risk Assessment Factors
The model considers five key dimensions that academic research and industry practice have identified as crucial for determining appropriate investment strategies:

a) **Investment Experience** (Scale 1-5)
- Measures familiarity with financial markets
- Higher experience suggests greater capacity to understand and accept investment risk
- Contributes positively to risk score

b) **Investment Horizon** (1-20 years)
- Longer horizons allow for more risk-taking due to:
 - More time to recover from market downturns
 - Greater ability to capture equity risk premium
- Normalized to 1-5 scale in calculations

c) **Income Stability** (Scale 1-5)
- Measures predictability of income streams
- Higher stability (5) enables more risk-taking because:
 - More reliable cash flows for living expenses
 - Less likely to need emergency liquidation of investments
 - Greater ability to weather market downturns
- Score is inverted in calculation as higher stability supports higher risk capacity

d) **Risk Tolerance** (Scale 1-5)
- Psychological ability to accept investment volatility
- Behavioral component distinct from risk capacity
- Direct measure of willingness to accept potential losses

e) **Investment Knowledge** (Scale 1-5)
- Understanding of financial markets and principles
- Higher knowledge correlates with better decision-making during market stress
- Contributes positively to risk score

### 2. Scoring Methodology

The risk score calculation involves:

1. **Normalization**:
  - Investment horizon converted from 1-20 years to 1-5 scale
  - Income stability inverted (5→1, 4→2, etc.) to align with risk capacity
  - Other factors used directly on 1-5 scale

2. **Weighted Average**:
  - Default weights:
    - Investment Experience: 20%
    - Investment Horizon: 20%
    - Income Stability: 20%
    - Risk Tolerance: 25%
    - Investment Knowledge: 15%
  - Weights are adjustable through interactive sliders
  - Sum of weights normalized to 100%

3. **Final Score**:
  - Weighted average rounded to nearest integer
  - Bounded between 1-5
  - Maps to risk profiles from Conservative (1) to Aggressive (5)

### 3. Asset Allocation Models

Risk scores map to these allocation models:

1. **Conservative (Score 1)**:
  - 20% Stocks, 70% Bonds, 10% Cash
  - Focus on capital preservation
  - Suitable for short horizons or very low risk tolerance

2. **Moderately Conservative (Score 2)**:
  - 40% Stocks, 55% Bonds, 5% Cash
  - Limited growth with stability
  - Suitable for shorter-medium horizons

3. **Moderate (Score 3)**:
  - 60% Stocks, 35% Bonds, 5% Cash
  - Balanced growth and stability
  - Suitable for medium-term horizons

4. **Moderately Aggressive (Score 4)**:
  - 75% Stocks, 20% Bonds, 5% Cash
  - Growth emphasis with some stability
  - Suitable for longer horizons

5. **Aggressive (Score 5)**:
  - 90% Stocks, 10% Bonds, 0% Cash
  - Maximum growth focus
  - Suitable for longest horizons and highest risk tolerance

### 4. Interactive Features

The implementation includes:
- Sliders for all input factors
- Adjustable factor weights
- Real-time updates of:
 - Risk score calculation
 - Profile description
 - Asset allocation visualization
- Detailed output showing calculation components

## Theoretical Foundation and Justification

This model builds on several key theoretical frameworks:

1. **Modern Portfolio Theory (MPT)**:
  - Asset allocations based on efficient frontier concepts
  - Risk-return tradeoff optimization
  - Diversification benefits

2. **Behavioral Finance**:
  - Recognition of psychological factors in risk tolerance
  - Separation of risk capacity from risk tolerance
  - Consideration of behavioral biases

3. **Life-Cycle Investing**:
  - Age and horizon considerations
  - Human capital factors (income stability)
  - Time diversification principles

4. **Financial Planning Best Practices**:
  - Holistic assessment of client situation
  - Multiple factor consideration
  - Regular rebalancing implied

## Limitations and Considerations

1. **Questionnaire Limitations**:
  - Self-reported data may be biased
  - Risk tolerance can change with market conditions
  - Simplified measures of complex characteristics

2. **Model Simplifications**:
  - Linear scoring model
  - Discrete risk profiles
  - Limited asset classes

3. **Implementation Considerations**:
  - Need for regular updates
  - Market condition impacts
  - Individual circumstances not captured

This implementation represents a simplified but practical approach to automated risk profiling and portfolio recommendation, helping to understand the workings of the **core robo-advisory concepts**.

In [1]:
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


The interactive nature of this example helps understand:

- How different factors affect risk profiling
- The relationship between risk scores and asset allocation
- The impact of different weights on the final recommendation
- The practical implementation of risk assessment in robo-advisors

In [3]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
class RiskProfiler:
    def __init__(self):
        """
        Initialize the Risk Profiler with questionnaire and risk profiles
        """
        # Define questions and their descriptions
        self.risk_questions = {
            'investment_experience': {
                'question': "Investment experience",
                'description': "Years of investment experience (1=Novice to 5=Expert)",
                'min': 1,
                'max': 5,
                'default': 3
            },
            'investment_horizon': {
                'question': "Investment horizon",
                'description': "Expected investment timeframe in years",
                'min': 1,
                'max': 20,
                'default': 10
            },
            'income_stability': {
                'question': "Income stability",
                'description': "1 (Very unstable) to 5 (Very stable)",
                'min': 1,
                'max': 5,
                'default': 3
            },
            'risk_tolerance': {
                'question': "Risk tolerance",
                'description': "1 (Very conservative) to 5 (Very aggressive)",
                'min': 1,
                'max': 5,
                'default': 3
            },
            'investment_knowledge': {
                'question': "Investment knowledge",
                'description': "1 (Beginner) to 5 (Expert)",
                'min': 1,
                'max': 5,
                'default': 3
            }
        }
        
        # Define risk profiles with more detailed descriptions
        self.risk_profiles = {
            1: {
                'name': "Conservative",
                'description': "Focus on preserving capital with minimal risk. Suitable for short-term goals or low risk tolerance."
            },
            2: {
                'name': "Moderately Conservative",
                'description': "Emphasis on stability with some growth potential. Suitable for medium-term goals with lower risk tolerance."
            },
            3: {
                'name': "Moderate",
                'description': "Balance between stability and growth. Suitable for medium to long-term goals with moderate risk tolerance."
            },
            4: {
                'name': "Moderately Aggressive",
                'description': "Emphasis on growth with acceptable risk. Suitable for long-term goals with higher risk tolerance."
            },
            5: {
                'name': "Aggressive",
                'description': "Maximum growth potential with higher risk tolerance. Suitable for long-term goals with highest risk tolerance."
            }
        }
        
        # Define asset allocation models
        self.allocation_models = {
            1: {'Stocks': 20, 'Bonds': 70, 'Cash': 10},
            2: {'Stocks': 40, 'Bonds': 55, 'Cash': 5},
            3: {'Stocks': 60, 'Bonds': 35, 'Cash': 5},
            4: {'Stocks': 75, 'Bonds': 20, 'Cash': 5},
            5: {'Stocks': 90, 'Bonds': 10, 'Cash': 0}
        }

class InteractiveRiskProfiler:
    def __init__(self):
        self.profiler = RiskProfiler()
        self.create_widgets()
        
    def create_widgets(self):
        """
        Create interactive widgets for risk profiling
        """
        style = {'description_width': 'initial'}
        layout = widgets.Layout(width='80%')
        
        # Create sliders for each question
        self.sliders = {}
        for key, info in self.profiler.risk_questions.items():
            self.sliders[key] = widgets.IntSlider(
                value=info['default'],
                min=info['min'],
                max=info['max'],
                description=f"{info['question']} ({info['description']})",
                style=style,
                layout=layout
            )
        
        # Create weight adjustment sliders
        self.weight_sliders = {}
        for key in self.profiler.risk_questions.keys():
            self.weight_sliders[key] = widgets.FloatSlider(
                value=0.2,
                min=0,
                max=1,
                step=0.05,
                description=f'Weight for {key}',
                style=style,
                layout=layout
            )
        
        # Create update button
        self.update_button = widgets.Button(
            description='Update Profile',
            button_style='primary'
        )
        self.update_button.on_click(self.update_profile)
        
        # Create output widget
        self.output = widgets.Output()
        
        # Display widgets
        display(widgets.VBox([
            widgets.HTML(value='<h3>Risk Assessment Questionnaire</h3>'),
            *self.sliders.values(),
            widgets.HTML(value='<h3>Factor Weights</h3>'),
            *self.weight_sliders.values(),
            self.update_button,
            self.output
        ]))
    
    def calculate_risk_score(self, responses, weights):
        """
        Calculate risk score with improved scoring methodology

        Higher values indicate higher risk tolerance/capacity EXCEPT for income stability
        where higher values (more stable income) should allow for more risk taking
        """
        # Normalize investment horizon to 1-5 scale
        normalized_horizon = np.interp(responses['investment_horizon'], 
                                     (1, 20),  # Original range
                                     (1, 5))    # Target range
        
        # Create normalized responses
        normalized_responses = responses.copy()
        normalized_responses['investment_horizon'] = normalized_horizon

         # Invert income stability score (5 becomes 1, 4 becomes 2, etc.)
        # This way, more stable income contributes to a higher risk capacity
        normalized_responses['income_stability'] = 6 - normalized_responses['income_stability']
    
        # Calculate weighted score
        weighted_score = sum(normalized_responses[q] * weights[q] for q in weights.keys())
        
        # Map to risk score (1-5)
        risk_score = int(round(weighted_score))
        
        return max(1, min(5, risk_score))
    
    def update_profile(self, _):
        """
        Update and display risk profile based on current widget values
        """
        # Get current responses and weights
        responses = {key: slider.value for key, slider in self.sliders.items()}
        weights = {key: slider.value for key, slider in self.weight_sliders.items()}
        
        # Normalize weights to sum to 1
        weight_sum = sum(weights.values())
        weights = {k: v/weight_sum for k, v in weights.items()}
        
        # Calculate risk score
        risk_score = self.calculate_risk_score(responses, weights)
        
        # Get recommendation
        profile = self.profiler.risk_profiles[risk_score]
        allocation = self.profiler.allocation_models[risk_score]
        
        # Display results
        with self.output:
            clear_output(wait=True)
            
            # Print detailed results
            print(f"\nRisk Assessment Results:")
            print(f"Risk Score: {risk_score} out of 5")
            print(f"\nRisk Profile: {profile['name']}")
            print(f"Description: {profile['description']}")

            print("\nFactors Considered:")
            print(f"- Investment Experience: {responses['investment_experience']}/5 (higher = more experience)")
            print(f"- Investment Horizon: {responses['investment_horizon']} years")
            print(f"- Income Stability: {responses['income_stability']}/5 (higher = more stable income = higher risk capacity)")
            print(f"- Risk Tolerance: {responses['risk_tolerance']}/5 (higher = more tolerance)")
            print(f"- Investment Knowledge: {responses['investment_knowledge']}/5 (higher = more knowledge)")
        
            print("\nInputs and Weights Used:")
            for key in responses.keys():
                print(f"{key}: {responses[key]} (weight: {weights[key]:.2f})")
            
            print("\nRecommended Asset Allocation:")
            for asset, percentage in allocation.items():
                print(f"{asset}: {percentage}%")
            
            # Plot allocation
            plt.figure(figsize=(10, 6))
            plt.pie(allocation.values(), 
                   labels=[f"{k}\n({v}%)" for k, v in allocation.items()],
                   autopct='%1.1f%%',
                   colors=sns.color_palette("pastel"))
            plt.title(f'Recommended Asset Allocation\nRisk Profile: {profile["name"]} (Score: {risk_score}/5)')
            plt.axis('equal')
            plt.show()

# Create and display interactive profiler
interactive_profiler = InteractiveRiskProfiler()